# Road Following - Live demo

### Load Trained Model

Load libraries

In [1]:
import torchvision
import torch
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
import time
from datetime import datetime, timedelta

from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg


Load pretrained model. It has 3 components: 1 encoder and 2 decoders.

In [3]:
encoder = EfficientNet.from_name('efficientnet-b0')
encoder._fc = torch.nn.Identity()
regressor = torch.nn.Linear(1280, 2)
classifier = torch.nn.Linear(1280, 2)

MODEL_PATH='multi_task_CNN/'
encoder.load_state_dict(torch.load(MODEL_PATH+'backbone_0.pth'))
regressor.load_state_dict(torch.load(MODEL_PATH+'head_xy_0.pth'))
classifier.load_state_dict(torch.load(MODEL_PATH+'head_stop_0.pth'))

# Transfer to GPU from CPU, use only half floats for efficiency
device = torch.device('cuda')
encoder = encoder.to(device)
encoder = encoder.eval().half()
regressor = regressor.to(device)
regressor = regressor.eval().half()
classifier = classifier.to(device)
classifier = classifier.eval().half()

### Creating the Pre-Processing Function

1. Convert from HWC layout to CHW layout
3. Transfer the data from CPU to GPU
4. Add a batch dimension

Note: EfficientNet has built in standardization for the inputs!

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    return image[None, ...]

Start camera, max FPS is 30. If the Python kernel is shut down without closing the camera instance, a system reset will be needed to use it again (see last cell of notebook).

In [5]:
camera = Camera(fps=30)

Create robot instance which will drive the motors.

In [30]:
from jetbot import Robot, bgr8_to_jpeg

robot = Robot()

Check that system time is correct! Docker uses universal time, so need to add 2h.

In [7]:
datetime.now()+timedelta(hours=2)

datetime.datetime(2021, 8, 13, 17, 34, 3, 96728)

Class logging the stops.

In [8]:
 
LOG_PATH = "/workspace/temp_logs/"
LOG_NAME = "stop_log.txt"

class StopLogger:
    def __init__(self, path=LOG_PATH+LOG_NAME, start_stop=0, n_stops=16, tz_offset=2):
        self.path=path
        self.n_stops=n_stops
        self.time_start="NaN"
        self.current_stop=start_stop-1
        self.tz_offset=tz_offset
    def start_stop(self):
        now=datetime.now()+timedelta(hours=self.tz_offset)
        self.time_start=now.strftime("%d/%m/%Y %H:%M:%S.%f")
        self.current_stop=(self.current_stop+1)%self.n_stops
    def log_stop(self):
        with open(self.path, 'a') as f:
            now=datetime.now()+timedelta(hours=self.tz_offset)
            line=self.time_start+";"+now.strftime("%d/%m/%Y %H:%M:%S.%f")+";"+str(self.current_stop)
            f.write(line+"\n")
            self.time_start="NaN"
        

The control loop methods.

In [46]:
angle = 0.0
angle_last = 0.0
angle_sum=0
frame_count=0

def process_frame(frame_dict, check_stop, P_thresh, base_speed, K_p, K_i, K_d, bias,
                  max_K_d, max_PID, logger, save_pics=False):
    '''
    frame_dict : dictionary with a frame from the camera
    angle : estimated angle to take (error)
    angle_last : estimated angle to take (error) in previous step
    angle_sum : accumulated angle (error) over all previous steps
    K_p : proportinal gain constant
    K_i : integral gain constant
    K_d : derivative gain constant
    bias : steady state bias term
    ---------------
    return : boolean indicating whether it stopped for measurements in this frame
    '''   
    global angle, angle_last, angle_sum, frame_count
    
    image = preprocess(frame_dict['new'])
    features=encoder(image)
    angle,xy=estimate_angle(features)
    
    PID=PID_control(angle, angle_last, angle_sum, K_p, K_i, K_d, bias, max_K_d, max_PID)
    angle_sum += angle
    angle_last = angle

    # Constrain speed to be [0,1] for both wheels
    robot.left_motor.value = max(min(base_speed + PID, 1.0), 0.0)
    robot.right_motor.value = max(min(base_speed - PID, 1.0), 0.0)
    
    if save_pics:
        save(frame_dict['new'],xy,frame_count)
        frame_count+=1
    
    if check_stop:
        P_move=estimate_P_move(features)
        if P_move < P_thresh:
            robot.stop()
            angle_last=0.0
            if logger is not None:
                logger.start_stop()
                print("stop nr: %d, P_move=%.4f"%(logger.current_stop,P_move))
                time.sleep(10)
                logger.log_stop()
            else:
                time.sleep(10)
            return True
    
    return False
        
        
def estimate_P_move(features):
    y = classifier(features)
    return float(F.softmax(y, dim=1).flatten()[0])

def estimate_angle(features):
    xy = regressor(features).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (xy[1]+0.5) / 2.0
    return np.arctan2(x, y), xy
    
def PID_control(angle, angle_last, angle_sum, K_p, K_i, K_d, bias, max_K_d, max_PID):
    
    # proportional control
    P= angle * K_p
    # integral control
    I = K_i * angle_sum + bias
    # derivative control
    D = max(min((angle - angle_last) * K_d, max_K_d),-max_K_d)
    # Put upper limit on steering from proportional and derivative control
    # to smooth turns.
    return max(min( P + D, max_PID),-max_PID) + I

def save(image,xy,count):
    snapshot = image.copy()
    snapshot = cv2.circle(snapshot, (int(xy[0]*112.0+112.0), int(xy[1]*112.0+112.0)), 8, (0, 0, 255), 3)
    image_path = os.path.join(DATASET_DIR, str(count) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(bgr8_to_jpeg(snapshot))
    
#execute({'new': camera.value})

Method for measuring how long the different parts of the control loop take.

In [10]:
def benchmark(frame_dict):
    t_start = time.time()
    image = preprocess(frame_dict['new'])
    t_pre = time.time()
    
    features=encoder(image)
    t_feat = time.time()
    xy = regressor(features).detach().float().cpu().numpy().flatten()
    t_xy = time.time()
    p=float(F.softmax(classifier(features), dim=1).flatten()[0])
    t_p = time.time()
    return {"t_pre":t_pre-t_start, "t_feat":t_feat-t_pre, "t_xy":t_xy-t_feat, "t_p":t_p-t_xy }
    

Method for testing the control loop without moving the robot. Used at every startup before starting operations to see if camera etc. is okay.

In [11]:
import IPython
def predict(change, K_p=0.1, K_i=0, K_d=0.1,
                         bias=0, max_K_d=0.7, max_PID=0.25):
    global out, angle, angle_last, angle_sum
    image = preprocess(change['new'])
    
    p = classifier(encoder(image))
    p=float(F.softmax(p, dim=1).flatten()[0])
    
    xy = regressor(encoder(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    y_alt=(xy[1] + 0.5) / 2.0
    angle_base=np.arctan2(x, xy[1])
    angle=np.arctan2(x, y)
    angle_alt=np.arctan2(x, y_alt)
    pid_base=PID_control(angle_base, angle_last, angle_sum, K_p, K_i, K_d, bias, max_K_d, max_PID)
    pid=PID_control(angle, angle_last, angle_sum, K_p, K_i, K_d, bias, max_K_d, max_PID)
    pid_alt=PID_control(angle_alt, angle_last, angle_sum, K_p, K_i, K_d, bias, max_K_d, max_PID)
    
    out0.update(IPython.display.Pretty('P: %.2f ; x: %.2f ; y: %.2f -> angle: %.2f ; pid: %.2f' % (p, x, xy[1], angle_base*180/3.14,pid_base)))
    out.update(IPython.display.Pretty('P: %.2f ; x: %.2f ; y: %.2f -> angle: %.2f ; pid: %.2f' % (p, x, y, angle*180/3.14,pid)))
    out2.update(IPython.display.Pretty('P: %.2f ; x: %.2f ; y: %.2f -> angle: %.2f ; pid: %.2f' % (p, x, y_alt, angle_alt*180/3.14,pid_alt)))
    

    

Code snippet used for calibrating the bias term for PID. This will move the robot!

In [12]:
'''
#old bias before fixed back-wheel was -0.0332
base_speed=0.25
bias= 0.03
robot.left_motor.value = base_speed + bias
robot.right_motor.value = base_speed - bias
time.sleep(3)
robot.stop()
'''

'\n#old bias before fixed back-wheel was -0.0332\nbase_speed=0.25\nbias= 0.03\nrobot.left_motor.value = base_speed + bias\nrobot.right_motor.value = base_speed - bias\ntime.sleep(3)\nrobot.stop()\n'

Widget to check what the robot sees. It displays the camera feed.

In [13]:
'''image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)'''

"image_widget = ipywidgets.Image()\n\ntraitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)\n\ndisplay(image_widget)"

Code snippet for initital check before starting to use the robot. It takes time to initialize the CNNs on the GPU, so wait until the display changes from "Start" to the estimated control values.

In [14]:
out0 = display(IPython.display.Pretty('Start'), display_id=True)
out = display(IPython.display.Pretty('Start'), display_id=True)
out2 = display(IPython.display.Pretty('Start'), display_id=True)
camera.observe(predict, names='value')


P: 1.00 ; x: 0.02 ; y: -0.18 -> angle: 175.30 ; pid: 0.25

P: 1.00 ; x: 0.02 ; y: 0.34 -> angle: 2.57 ; pid: 0.01

P: 1.00 ; x: 0.02 ; y: 0.16 -> angle: 5.53 ; pid: 0.02

If estimates look fine, turn it off with camera.unobserve() so the actual deployment can be started.

In [15]:
camera.unobserve(predict, names='value')

### Robot deployment

Set parameters, initialize logger object.

In [61]:
actions_before_break=15
break_s=0.2
has_stopped=False
check_stop=True
counter=0
base_speed=0.24
bias=0.03
start_boost=0.08
start=True
logger=StopLogger(start_stop=9)

Start moving the robot. The outer loop defines how long the robot should operate. Every iteration is one cycle with "actions_before_break" frames processed. Finishing one round around ESAT 00.08 takes around 70 iterations.

In [62]:
for i in range(400):
    for i in range(actions_before_break):
        # extra speed when starting to overcome static friction
        if start:
            base_speed_adj=base_speed+start_boost
            start=False
        else:
            base_speed_adj=base_speed
        # if robot stops to take measurements do not stop again for m cycles
        if process_frame({'new': camera.value},check_stop, P_thresh=0.015,
                         base_speed=base_speed_adj, K_p=0.17, K_i=0, K_d=0.2,
                         bias=bias, max_K_d=0.4, max_PID=0.33, logger=logger):
            check_stop=False
            counter=0
            start=True
        else:
            counter+=1
        if counter > 30:
            check_stop=True

    robot.stop()
    angle_last=0.0
    time.sleep(break_s)
    start=True
print("done")   

stop nr: 9, P_move=0.0011
stop nr: 10, P_move=0.0023
stop nr: 11, P_move=0.0038
stop nr: 12, P_move=0.0010
stop nr: 13, P_move=0.0000
stop nr: 14, P_move=0.0000
stop nr: 15, P_move=0.0000
stop nr: 0, P_move=0.0001
stop nr: 1, P_move=0.0002
stop nr: 2, P_move=0.0003
stop nr: 3, P_move=0.0016
stop nr: 4, P_move=0.0003
stop nr: 5, P_move=0.0001
stop nr: 6, P_move=0.0000
stop nr: 7, P_move=0.0000
stop nr: 8, P_move=0.0002
stop nr: 9, P_move=0.0138
stop nr: 10, P_move=0.0050
stop nr: 11, P_move=0.0001
stop nr: 12, P_move=0.0005
stop nr: 13, P_move=0.0000
stop nr: 14, P_move=0.0002
stop nr: 15, P_move=0.0000
stop nr: 0, P_move=0.0006
stop nr: 1, P_move=0.0015
stop nr: 2, P_move=0.0068
stop nr: 3, P_move=0.0014
done


If robot control loop needs to be terminated early, the motors will continue at their last speed setting. To stop the motors robot.stop() needs to be used.

In [45]:
robot.stop()

### Benchmarking

In [27]:
time_sum_dict={"t_pre":0, "t_feat":0, "t_xy":0, "t_p":0 }
total_time_sum=0
for i in range(100):
    time_dict=benchmark({'new': camera.value})
    for key in time_dict.keys():
        time_sum_dict[key]+=time_dict[key]
        total_time_sum+=time_dict[key]

print("preprocessing: {:.5f}s, {:.3f} of total".format(time_sum_dict["t_pre"]/1000,time_sum_dict["t_pre"]/total_time_sum))
print("feature extraction: {:.5f}s, {:.3f} of total".format(time_sum_dict["t_feat"]/1000,time_sum_dict["t_feat"]/total_time_sum))
print("regression: {:.5f}s, {:.3f} of total" .format(time_sum_dict["t_xy"]/1000,time_sum_dict["t_xy"]/total_time_sum))
print("classification: {:.5f}s, {:.3f} of total".format(time_sum_dict["t_p"]/1000,time_sum_dict["t_p"]/total_time_sum))

preprocessing: 0.00041s, 0.047 of total
feature extraction: 0.00828s, 0.933 of total
regression: 0.00010s, 0.011 of total
classification: 0.00008s, 0.009 of total


### Code for recording what the robot sees + marking predictions.

In [47]:
import os
DATASET_DIR = 'demo'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')
actions_before_break=15
break_s=0.2
has_stopped=False
check_stop=True
counter=0
base_speed=0.22
bias=0.03
start_boost=0.08
start=True
logger=None

In [48]:
for i in range(100):
    for i in range(actions_before_break):
        # extra speed when starting to overcome static friction
        if start:
            base_speed_adj=base_speed+start_boost
            start=False
        else:
            base_speed_adj=base_speed
        # if robot stops to take measurements do not stop again for m cycles
        if process_frame({'new': camera.value},check_stop, P_thresh=0.015,
                         base_speed=base_speed_adj, K_p=0.17, K_i=0, K_d=0.2,
                         bias=bias, max_K_d=0.4, max_PID=0.33, logger=None, save_pics=True):
            check_stop=False
            counter=0
            start=True
        else:
            counter+=1
        if counter > 30:
            check_stop=True

    robot.stop()
    angle_last=0.0
    time.sleep(break_s)
    start=True
print("done")   
!zip -r -q demo.zip {DATASET_DIR}

done


### CAMERA NEEDS TO BE CLOSED
If the Python kernel is shut down without closing the camera instance, a system reset will be needed to use it again.

In [28]:
camera.stop()